In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **기본 설정**

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

# **Application train & test 데이터 EDA**

In [ ]:
#데이터 불러온 후 전체적인 행과 열의 수와 데이터 확인
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

print('Training data shape: ',  app_train.shape)
print('Testing data shape: ',  app_test.shape)

In [ ]:
app_train.head()

In [ ]:
app_test.head()

* **결측치 확인**

In [ ]:
app_train.isnull().sum() 

In [ ]:
app_test.isnull().sum() 

In [ ]:
# Null 값 확인 가능한 사용자 함수 (데이터에서 비율)
# 결측치 있는 데이터의 개수를 센 후, 시각화

def nulldata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    f,ax =plt.subplots(figsize=(10,5))
    plt.xticks(rotation='90')
    fig=sns.barplot(ms.index, ms["Percent"],color="green",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of null values', fontsize=15)
    plt.title('Percent null data by feature', fontsize=15)
    print(ms.count())

In [ ]:
# app_train 데이터 확인
# 전체 122개 중 67개 결측치
nulldata(app_train)

In [ ]:
# app_test 데이터 확인
# 전체 121개 중 64개 결측치
nulldata(app_test)

* **Train data의 Target 비율 확인**

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
#target의 비율 확인
#307511명 중 282686명이 대출을 제때 상환
app_train['TARGET'].value_counts()/app_train.shape[0]

In [ ]:
#히스토그램으로 target 시각화
app_train['TARGET'].astype(int).plot.hist();

결과 해석: 대출을 상환할 수 있는 사람이 그렇지 못한 사람보다 훨씬 많다

이는 불균형한 데이터이므로 이후에 조정이 필요하다

* **AMT_CREDIT(대출금액) 분포 확인**

In [ ]:
# AMT_CREDIT 칼럼 확인
app_train['AMT_CREDIT'].head()

In [ ]:
# AMT_CREDIT을 hist()를 통해 시각화해서 파악
app_train['AMT_CREDIT'].hist()

In [ ]:
# AMT_CREDIT을 박스플롯을 통해 시각화해서 파악
sns.boxplot(app_train['AMT_CREDIT'])

In [ ]:
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

In [ ]:
# 중요 변수들 간의 상관관계 파악
col = app_train[columns].corr()

plt.figure(figsize = (20, 10))
sns.heatmap(col)

# **각 변수별로 시각화하기**
**1. AMT_CREDIT 시각화**

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1) # target이 1일 때
    cond_0 = (df['TARGET'] == 0) # target이 0일 때
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False) # plot 생성
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1]) # target 1일 때
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=ax[0][1]) # target 2일 때

In [ ]:
# AMT_CREDIT 확인

sns.boxplot(app_train['AMT_CREDIT'])    #이상치가 많음

In [ ]:
# 이상치를 포함하지 않도록 조건 설정해 준 후 distplot을 통해 시각화
condition_1 = app_train['AMT_CREDIT'] < 60000
condition_2 = app_train['AMT_CREDIT'] > 48000
sns.distplot(app_train[condition_1 & condition_2]['AMT_CREDIT'])

In [ ]:
# TARGET 값에 따른 AMT_CREDIT 분포 비교
# 같은 공간에 시각화해서 비교
cond1 = (app_train['TARGET'] == 1)
cond0 = (app_train['TARGET'] == 0)

fig, ax = plt.subplots(figsize = (12, 4), nrows = 1, ncols = 2, squeeze = False)

sns.violinplot(x = 'TARGET', y = 'AMT_CREDIT', data = app_train[condition_1 & condition_2], ax = ax[0][0])
sns.distplot(app_train[cond0 & condition_1 & condition_2]['AMT_CREDIT'], label = '0', color = 'blue')
sns.distplot(app_train[cond1 & condition_1 & condition_2]['AMT_CREDIT'], label = '1', color = 'red')

In [ ]:
## 상관관계 확인 결과 상관관계 없음
app_train[['TARGET', 'AMT_CREDIT']].corr()

**2. EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 시각화**

In [ ]:
column2 = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
show_hist_by_target(app_train, column2)

히스토그램 시각화 결과, EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3에서 음의 상관관계가 뚜렷하게 나타남

In [ ]:
# 각 EXT_SOUCE를 TARGET별로 나눠서 시각화
plt.figure(figsize=(10,12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)

kdeplot을 통해 각 변수별로 TARGET 값을 확인한 결과, 모두 음의 상관관계가 있는 것으로 보인다는 것을 알 수 있음

In [ ]:
# TARGET 변수와 EXT_SOURCE/DAYS_BIRTH와의 상관관계와 EXT_SOURCE 서로간의 상관관계를 살핌


ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

#상관관계를 히트맵으로 확인
#상관관계 확인 결과 세 개의 속성 모두 target과 약한 음의 상관관계
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

**3. DAYS_BIRTH 시각화**

In [ ]:
# DAYS_BIRTH를 절대값으로 바꿔줌
# DAYS_BIRTH 데이터 확인
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])

sns.distplot(app_train['DAYS_BIRTH']) #고르게 분포

In [ ]:
#고객 나이를 TARGET별로 나눠서 시각화

plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

나이가 어릴수록 대출을 상환하지 못할 가능성이 높을 것

In [ ]:
# 상관관계 확인 결과 상관관계 약간 있음
app_train[['TARGET', 'DAYS_BIRTH']].corr()

**4. AMT_GOODS_PRICE 시각화**

In [ ]:
app_train['AMT_GOODS_PRICE'].hist()

In [ ]:
# 이상치를 포함하지 않도록 조건 설정해 준 후 distplot을 통해 시각화
condi_1 = app_train['AMT_GOODS_PRICE'] < 65000
condi_2 = app_train['AMT_GOODS_PRICE'] > 40000
sns.distplot(app_train[condi_1 & condi_2]['AMT_GOODS_PRICE'])

In [ ]:
# TARGET 값에 따른 AMT_GOODS_PRICE 분포 비교
# 같은 공간에 시각화해서 비교
cond1 = (app_train['TARGET'] == 1)
cond0 = (app_train['TARGET'] == 0)

fig, ax = plt.subplots(figsize = (12, 4), nrows = 1, ncols = 2, squeeze = False)

sns.violinplot(x = 'TARGET', y = 'AMT_GOODS_PRICE', data = app_train[condi_1 & condi_2], ax = ax[0][0])
sns.distplot(app_train[cond0 & condi_1 & condi_2]['AMT_GOODS_PRICE'], label = '0', color = 'blue')
sns.distplot(app_train[cond1 & condi_1 & condi_2]['AMT_GOODS_PRICE'], label = '1', color = 'red')

In [ ]:
# 상관관계 확인 결과 상관관계 거의 없음
app_train[['TARGET', 'AMT_GOODS_PRICE']].corr()

**5. NAME_FAMILY_STATUS 시각화**

In [ ]:
# 데이터 타입이 객체인 컬럼들을 object_columns로 설정
# 이 중 가족 유형, 성별 선택
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
#시각화를 위한 사용자 정의 함수

def plot_stats(feature,label_rotation=False,horizontal_layout=True):
    temp = app_train[feature].value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Number of contracts': temp.values})

    # Calculate the percentage of target=1 per category value
    cat_perc = app_train[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)
    
    if(horizontal_layout):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,14))
    sns.set_color_codes("pastel")
    s = sns.barplot(ax=ax1, x = feature, y="Number of contracts",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    s = sns.barplot(ax=ax2, x = feature, y='TARGET', order=cat_perc[feature], data=cat_perc)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)

    plt.ylabel('Percent of target with value 1 [%]', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=10)

    plt.show();

In [ ]:
plot_stats('NAME_FAMILY_STATUS', True, True)

가족 유형 중에는 결혼한 사람이 가장 많았다
대출을 상환하지 않을 확률은 civil marriage, single/not married, sparated, married 순으로 높았다
결혼한 사람이 많은 것에 비해, 결혼한 사람들은 대출을 상대적으로 잘 상환하는 모습을 보여준다

**6. CODE_GENDER 시각화**

In [ ]:
plot_stats('CODE_GENDER')

In [ ]:
# 타겟 조건 설정 후, 타겟과 성별 value_counts()
# 대출 횟수 대비 연체 비율이 남성이 여성보다 높음 

cond_1 = (app_train['TARGET'] == 1)
cond_0 = (app_train['TARGET'] == 0)
print(app_train['CODE_GENDER'].value_counts()/app_train.shape[0])
print('\n연체인 경우\n',app_train[cond_1]['CODE_GENDER'].value_counts()/app_train[cond_1].shape[0])
print('\n연체가 아닌 경우\n',app_train[cond_0]['CODE_GENDER'].value_counts()/app_train[cond_0].shape[0])

전체적으로 여성이 대출한 비율이 더 높지만, 대출 비율에 비해 남성은 대출을 상환하지 못 한 경우가 많았음

# **train/test 데이터 합치기**
이후 데이터 분석을 위해 합쳐야 한다

In [ ]:
# concat을 이용해 train 데이터와 test 데이터 합쳐주고, shape로 확인
apps = pd.concat([app_train, app_test])
apps.shape

In [ ]:
# value_counts()를 통해 결합한 데이터의 값 확인
# 대출을 상환한 사람이 더 많고, NAN 값 존재
apps['TARGET'].value_counts(dropna = False)

# **Feature Engineering**
* **결측치 처리**

In [ ]:
# 레코드단 mean과 std 만 구함
# max, min 도 해볼 수 있지만, 상관관계가 높은 것에 대해 너무 가공하면 성능이 오히려 떨어질 수 있으므로 mean과 std 만 구함
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

#결측치 확인
#40950의 결측치가 있음
apps['APPS_EXT_SOURCE_STD'].isnull().value_counts()

In [ ]:
# mean으로 결측치 채우기
apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

#결측치 확인
#결측치 없음
apps['APPS_EXT_SOURCE_STD'].isnull().value_counts()

* **feature 가공하기**

In [ ]:
# AMT_CREDIT 비율로 Feature 가공

apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']
apps['APPS_CREDIT_BIRTH_RATIO'] = apps['DAYS_BIRTH'] / apps['AMT_CREDIT']
apps['APPS_INCOME_CREDIT_DIFF'] = apps['AMT_INCOME_TOTAL'] - apps['AMT_CREDIT']

# DAYS_BIRTH 비율로 Feature 가공

apps['APPS_CREDIT_BIRTH_RATIO'] = apps['AMT_CREDIT'] / apps['DAYS_BIRTH']
apps['APPS_GOODS_BIRTH_RATIO'] = apps['AMT_GOODS_PRICE'] / apps['DAYS_BIRTH']

# AMT_INCOME_TOTAL 비율로 대출 금액 관련 피처 가공

apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']

In [ ]:
apps[['APPS_ANNUITY_CREDIT_RATIO', 'APPS_GOODS_CREDIT_RATIO', 'APPS_CREDIT_GOODS_DIFF', 
     'APPS_CREDIT_BIRTH_RATIO', 'APPS_INCOME_CREDIT_DIFF', 'APPS_INCOME_EMPLOYED_RATIO',
     'APPS_CREDIT_BIRTH_RATIO', 'APPS_GOODS_BIRTH_RATIO',
     'APPS_CREDIT_INCOME_RATIO', 'APPS_GOODS_INCOME_RATIO']].head()

In [ ]:
#shape 확인
print('Training data shape: ', app_train.shape)
print('Testing data shape: ', app_test.shape)
print('apps data shape: ', apps.shape)

* **Label Encoding**

In [ ]:
# 데이터 확인
app_train.info()

In [ ]:
# 데이터 타입 확인
apps.dtypes.index

In [ ]:
# for문과 factorize를 이용해 한꺼번에 label encoding
object_columns = apps.dtypes[apps.dtypes == 'object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

In [ ]:
# object가 없어진 것을 확인
apps.info()

* **합쳤던 트레이닝 데이터와 테스트 데이터 다시 분리**

In [ ]:
#train 데이터에는 결측치를 제외한 데이터, test 데이터에는 결측치 데이터로 분리
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
print(apps.shape, apps_train.shape, apps_test.shape)

# **학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행**

In [ ]:
#피쳐용 데이터와 타겟 데이터 분리
ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis = 1)
target_app = apps_train['TARGET']

In [ ]:
#학습용/검증용 데이터 셋 분리
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size = 0.3, random_state = 2020)
print(train_x.shape, valid_x.shape)

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

* **Feature Importance 시각화**

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

# **학습된 Classifier를 이용하여 테스트 데이터 예측하고 결과를 Kaggle로 Submit**

In [ ]:
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]

app_test['TARGET'] = preds
app_test['TARGET'].head()

In [ ]:
app_test[['SK_ID_CURR', 'TARGET']].to_csv('apps_baseline_02.csv', index=False)

In [ ]:
!ls